In [ ]:
from pandas import read_csv
from autoencoder.autoencoder import AnomalyDetector
from turtleIsolationForests.preprocessFeatures import minmax_preprocess_features
from turtleIsolationForests.printResults import calc_confusion
from tensorflow.keras.losses import mae
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
train_dataframe = read_csv("./eda_simple_classification/network_data_mod_train.csv", index_col=0)
test_dataframe = read_csv("./eda_simple_classification/network_data_mod_test.csv", index_col=0)
X_train, X_test, train_labels, test_labels = minmax_preprocess_features(train_dataframe, test_dataframe)
X_train_ae = X_train[train_labels]

In [ ]:
autoenc = AnomalyDetector()
autoenc.compile(optimizer='adam', loss='mae')

In [ ]:
losses = []
val_losses = []
test_losses = []

In [ ]:
i = 0
while i < 5000:
    history = autoenc.fit(X_train_ae, X_train_ae, initial_epoch=i, epochs=i+1, validation_split=0.2, shuffle=True)
    reconstructions = autoenc.predict(X_test)
    test_loss = mae(X_test, reconstructions)
    losses.append(history.history["loss"][0])
    val_losses.append(history.history["val_loss"][0])
    test_losses.append(np.mean(test_loss))
    i += 1

In [ ]:
plt.plot(losses)
plt.plot(val_losses)
plt.plot(test_losses)
plt.title("Training, Validation, and Test Loss over epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim([0, .08])
plt.legend(["Loss", "Validation Loss", "Test Loss"])
plt.show()